In [8]:
import numpy as np


from llmize import OPRO
import llmize
import os


In [5]:
def lp_maximization_problem(x):
    """
    Linear Programming maximization problem with penalty constraints.
    
    Args:
        x (list or numpy array): Decision variables [x1, x2, x3]
    
    Returns:
        float: Penalized objective function value (to be maximized)
    """
    x1, x2, x3 = x
    # Objective Function (Maximize Z)
    Z = 3*x1 + 4*x2 + 6*x3  # No negation since we maximize directly

    # Define a large penalty for constraint violations
    penalty = 0
    large_penalty = -1e2  # Large NEGATIVE penalty (to push infeasible solutions down)

    # Constraints
    if 2*x1 + 3*x2 + x3 > 15:  # Resource 1 constraint
        penalty += large_penalty
    if x1 + 2*x2 + 3*x3 > 20:  # Resource 2 constraint
        penalty += large_penalty
    if 4*x1 + x2 + 2*x3 > 16:  # Resource 3 constraint
        penalty += large_penalty
    if x1 < 0 or x2 < 0 or x3 < 0:  # Non-negativity constraints
        penalty += large_penalty

    return Z + penalty  # Maximize Z while discouraging violations

In [7]:
# Generate random solutions (list of lists) and scores (list of floats)
# Generate initial random solutions
num_samples = 16  # Batch size
num_vars = 3  # Number of decision variables
#round to 2 decimal places
init_samples = [np.round(np.random.uniform(0, 10, num_vars), 2) for _ in range(num_samples)]

# Calculate scores for initial solutions, round to 2 decimal places
init_scores = [np.round(lp_maximization_problem(x), 2) for x in init_samples]

print(init_samples)
print(init_scores)

[array([9.93, 3.9 , 7.87]), array([7.37, 0.18, 2.79]), array([6.41, 2.78, 7.47]), array([4.73, 0.72, 5.33]), array([0.7 , 4.36, 2.83]), array([4.44, 4.14, 1.46]), array([6.39, 1.13, 1.88]), array([1.25, 9.61, 1.73]), array([7.52, 4.78, 7.49]), array([5.18, 7.58, 0.04]), array([6.86, 2.52, 8.35]), array([0.55, 4.25, 8.77]), array([8.95, 1.56, 6.14]), array([9.57, 1.57, 4.74]), array([1.63, 7.29, 7.11]), array([1.78, 3.03, 8.21])]
[np.float64(-207.39), np.float64(-160.43), np.float64(-224.83), np.float64(-250.95), np.float64(-63.48), np.float64(-161.36), np.float64(-165.03), np.float64(-247.43), np.float64(-213.38), np.float64(-253.9), np.float64(-219.24), np.float64(-228.73), np.float64(-230.07), np.float64(-236.57), np.float64(-223.29), np.float64(-233.28)]


In [10]:
with open("lp_problem.txt", "r") as f:
    problem_text = f.read()

# Initialize the OPRO optimizer
opro = OPRO(problem_text=problem_text, obj_func=lp_maximization_problem,
            llm_model="gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY"))

prompt = opro.get_sample_prompt(init_samples=init_samples, init_scores=init_scores, optimization_type="maximize")
response = opro.get_sample_response(prompt)

llmize.utils.pretty_print(prompt=prompt, response=response)

Prompt:
Problem: Linear Programming Optimization
-----------------------------------------------------
Objective: Maximize the function
    Z = 3x1 + 4x2 + 6x3

Subject to constraints:
    2x1 + 3x2 + x3 ≤ 15
    x1 + 2x2 + 3x3 ≤ 20
    4x1 + x2 + 2x3 ≤ 16
    x1, x2, x3 ≥ 0

Below are some examples of solutions and their scores:

<sol> 9.93,3.9,7.87 <\sol>
score: -207.39

<sol> 7.37,0.18,2.79 <\sol>
score: -160.43

<sol> 6.41,2.78,7.47 <\sol>
score: -224.83

<sol> 4.73,0.72,5.33 <\sol>
score: -250.95

<sol> 0.7,4.36,2.83 <\sol>
score: -63.48

<sol> 4.44,4.14,1.46 <\sol>
score: -161.36

<sol> 6.39,1.13,1.88 <\sol>
score: -165.03

<sol> 1.25,9.61,1.73 <\sol>
score: -247.43

<sol> 7.52,4.78,7.49 <\sol>
score: -213.38

<sol> 5.18,7.58,0.04 <\sol>
score: -253.90

<sol> 6.86,2.52,8.35 <\sol>
score: -219.24

<sol> 0.55,4.25,8.77 <\sol>
score: -228.73

<sol> 8.95,1.56,6.14 <\sol>
score: -230.07

<sol> 9.57,1.57,4.74 <\sol>
score: -236.57

<sol> 1.63,7.29,7.11 <\sol>
score: -223.29

<sol> 1.78

In [ ]:
from llmize.callbacks import EarlyStopping, AdaptTempOnPlateau, OptimalScoreStopping

# Define the early stopping callback
earlystop_callback = EarlyStopping(monitor='best_score', min_delta=0.001, patience=50, verbose=1)

# Define the optimal score stopping callback
optimal_score_callback = OptimalScoreStopping(optimal_score=41.08, tolerance=0.01)

# Define the temperature adaptation callback
adapt_temp_callback = AdaptTempOnPlateau(monitor='best_score', init_temperature=1.0, min_delta=0.001, patience=20, factor=1.1, max_temperature=1.9, verbose=1)

callbacks = [earlystop_callback, optimal_score_callback, adapt_temp_callback]

In [ ]:
results = opro.maximize(init_samples=init_samples, init_scores=init_scores, num_steps=250, batch_size=16, callbacks=callbacks)
